Sebastien Chassot <sebastien.chassot@etu.hesge.ch><br>
_(GnuPG 0x3F432F8E)_ <br>
HES-SO hepia ITI <br>
[http://hepia.hesge.ch](http://hepia.hesge.ch)<br>

* * * 
***

# Test



In [2]:
%%bash
netstat -ant | grep 1234

tcp        0      0 0.0.0.0:1234            0.0.0.0:*               LISTEN     


## Creation du system de fichier minix ##

On créé un système de fichier que l'on format.<br>

On monte le système de fichier afin d'écrire une serie de fichier


In [36]:
%%bash

truncate -s 20M ./filesystems/new.minixfs.img
mkfs.minix -1 ./filesystems/new.minixfs.img
#mount -o loop -t minix ./filesystems/new.minixfs.img /mnt
if mount|grep mini;
    then
    for i in {1..50}; do touch /mnt/file_$i;done
    
    echo "Un contenu ...avec" > /mnt/file_14
    echo -e "\n\nStat sur un fichier : \n"
    stat /mnt/file_14
    echo
    #ls -al /mnt
else
    echo "not mounted"
fi

6848 inodes
20480 blocks
Firstdatazone=220 (220)
Zonesize=1024
Maxsize=268966912

/home/seba/documents/hepia/tp-minixfs/filesystems/new.minixfs.img on /mnt type minix (rw,relatime)


Stat sur un fichier : 

  File: ‘/mnt/file_14’
  Size: 19        	Blocks: 2          IO Block: 1024   regular file
Device: 702h/1794d	Inode: 15          Links: 1
Access: (0644/-rw-r--r--)  Uid: ( 1000/    seba)   Gid: ( 1000/    seba)
Access: 2015-06-06 19:46:39.000000000 +0200
Modify: 2015-06-06 19:46:39.000000000 +0200
Change: 2015-06-06 19:46:39.000000000 +0200
 Birth: -



## Verification de l'état du disque ##

on fait un hexdump de certain block


<br>

In [37]:
%%python2
from constantes import *
import sys
import constantes
import bloc_device as blk
import hexdump
import os
import sys

testfile="filesystems/new.minixfs.img"

disk = blk.bloc_device(BLOCK_SIZE, testfile)

hexdump.hexdump(disk.read_bloc(221))
del disk

00000000: 20 00 66 69 6C 65 5F 33  31 00 00 00 00 00 00 00   .file_31.......
00000010: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000020: 21 00 66 69 6C 65 5F 33  32 00 00 00 00 00 00 00  !.file_32.......
00000030: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000040: 22 00 66 69 6C 65 5F 33  33 00 00 00 00 00 00 00  ".file_33.......
00000050: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000060: 23 00 66 69 6C 65 5F 33  34 00 00 00 00 00 00 00  #.file_34.......
00000070: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000080: 24 00 66 69 6C 65 5F 33  35 00 00 00 00 00 00 00  $.file_35.......
00000090: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000000A0: 25 00 66 69 6C 65 5F 33  36 00 00 00 00 00 00 00  %.file_36.......
000000B0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000000C0: 26 00 66 69 6C 65 5F 33  37 00 00 00 00 00 00 00  &.file_37.......

INFO:file system opened successfully
INFO:file system closed


## Test d'effacement d'une entrée

On efface une entrée puis on remarque que l'inode de l'entrée vaut maintenant 0x00


### On sature le disque avec balloc appelé en boucle ###


<br>


In [38]:
%%python2
testfile="filesystems/new.minixfs.img"
import minixfs as mx
import hexdump
minixfs=mx.minix_file_system(testfile)

minixfs.del_entry(minixfs.inodes_list[1], "file_38")
hexdump.hexdump(minixfs.disk.read_bloc(221)[192:256])
#print(minixfs.lookup_entry(minixfs.inodes_list[1], 'file_1'))
for i in range(100000):
    minixfs.balloc()
    
del minixfs

00000000: 26 00 66 69 6C 65 5F 33  37 00 00 00 00 00 00 00  &.file_37.......
00000010: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000020: 00 00 66 69 6C 65 5F 33  38 00 00 00 00 00 00 00  ..file_38.......
00000030: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................


INFO:file system opened successfully
ERROR:Error no space left on device
Traceback (most recent call last):
  File "<stdin>", line 10, in <module>
  File "minixfs.py", line 129, in balloc
    raise BlockAllocError("Error no space left on device")
minixfs.BlockAllocError: Error no space left on device
INFO:file system closed


## Test allocation d'un block doublement indirect

On utilise ialloc_bloc() pour créer et écrire dans un block doublement indirect


<br>


In [54]:
%%python2
testfile="filesystems/new.minixfs.img"
import minixfs as mx
import hexdump

filename="file_127"
minixfs=mx.minix_file_system(testfile)


minixfs.add_entry(minixfs.inodes_list[1], filename, minixfs.balloc())
inode = minixfs.lookup_entry(minixfs.inodes_list[1], filename)
print(inode)
minixfs.ialloc_bloc(minixfs.inodes_list[inode], 500)

hexdump.hexdump(minixfs.disk.read_bloc(minixfs.bmap(minixfs.inodes_list[inode]), 5000))
#print(minixfs.lookup_entry(minixfs.inodes_list[1], 'file_1'))

del minixfs

224


INFO:file system opened successfully
Traceback (most recent call last):
  File "<stdin>", line 12, in <module>
  File "minixfs.py", line 279, in ialloc_bloc
    if not struct.unpack_from('H', self.disk.read_bloc(inode.i_dbl_indr_zone), (blk-7-MINIX_ZONESZ)/MINIX_ZONESZ):
struct.error: unpack_from requires a buffer of at least 2 bytes
INFO:file system closed


## Teste bmap ##

#####TODO #####
Verifier ce que renvoi bmap quand block inexistant. Est-ce que la taille change et comment faire?

bmap doit peut-être tester si bmap dépasse la taille spécifiée dans l'inode?

In [43]:
%%python2
import scapy
from scapy.all import *
from scapy.utils import rdpcap

pkts=rdpcap("./docs/tcpdump_hepia.cap")

for i in [3, 5, 226, 227]:
    print("\n\nPacket : "+ i.__str__()+ "\n")
    hexdump(pkts[i])
    print("\n")
    pkts[i].show2()




Packet : 3

0000   FC 94 E3 A8 4A 80 F4 6D  04 94 18 D2 08 00 45 00   ....J..m......E.
0010   00 48 F2 B7 40 00 40 06  4D 0A C0 A8 00 12 81 C2   .H..@.@.M.......
0020   B8 71 86 56 04 D2 7E 31  8A 76 05 88 91 9B 80 18   .q.V..~1.v......
0030   00 E5 26 B2 00 00 01 01  08 0A 00 61 78 C5 0D 15   ..&........ax...
0040   74 AA 76 76 76 76 00 00  00 00 F8 E1 40 73 00 00   t.vvvv......@s..
0050   04 00 00 00 04 00                                  ......


###[ Ethernet ]###
  dst       = fc:94:e3:a8:4a:80
  src       = f4:6d:04:94:18:d2
  type      = 0x800
###[ IP ]###
     version   = 4L
     ihl       = 5L
     tos       = 0x0
     len       = 72
     id        = 62135
     flags     = DF
     frag      = 0L
     ttl       = 64
     proto     = tcp
     chksum    = 0x4d0a
     src       = 192.168.0.18
     dst       = 129.194.184.113
     \options   \
###[ TCP ]###
        sport     = 34390
        dport     = search_agent
        seq       = 2117175926
        ack       = 92836251
     

In [47]:
!whois 129.194.184.113 |grep -3 inetnum


% No abuse contact registered for 129.194.0.0 - 129.194.255.255

inetnum:        129.194.0.0 - 129.194.255.255
netname:        UNIGE-CENTER
descr:          University of Geneva-Ctr
descr:          Geneva, Switzerland


In [75]:
%%python2
import minixfs as mx
help(mx)

Help on module minixfs:

NAME
    minixfs - This is the main minixfs API

FILE
    /home/seba/documents/hepia/tp-minixfs/minixfs.py

CLASSES
    __builtin__.object
        minix_file_system
    exceptions.Exception(exceptions.BaseException)
        MyBaseException
            AddEntryError
            AddError
            BlockAllocError
            DelEntryError
            DirFullError
            FileNameError
            FileNotFoundError
            InodeError
            OutboundError
    
    class AddEntryError(MyBaseException)
     |  Method resolution order:
     |      AddEntryError
     |      MyBaseException
     |      exceptions.Exception
     |      exceptions.BaseException
     |      __builtin__.object
     |  
     |  Methods inherited from MyBaseException:
     |  
     |  __init__(self, message)
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors inherited from MyBaseException:
     |  
     |  __weakref_